In [33]:
import pandas as pd
from pandas import Series
import numpy as np
import os
import pickle
import random
from collections import Counter
from operator import itemgetter
import joblib
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from scipy.stats import rankdata
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolToSmiles, MolFromSmiles
from rdkit.Chem import Lipinski, Descriptors
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import tmap as tm
from faerun import Faerun
from map4 import MAP4Calculator

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [34]:
folder = "/data/NPAtlas/"

# TEST with Epicospirocins, penicimeroterpenoids, and Phakefustatin A + SI structures
We considered the class *bacterium* to be the positive class and the class *fungus* to be the negative one. 

In [35]:
epicospirocins_smiles_list = ["OC1=C(O)C(O)=C(C(OC)OC23CC(C(O)=C(O)C(O)=C4C)=C4CO3)C2=C1C", \
          "OC1=C(O)C(C)=C(C(OC)OC23CC(C(O)=C(O)C(O)=C4C)=C4CO3)C2=C1O", \
          "OC5=C(O)C(O)=C(C(O)OC67CC(C(O)=C(O)C(O)=C8C)=C8CO7)C6=C5C", \
          "OC9=C(O)C(C)=C(C(O)OC%10%11CC(C(O)=C(O)C(O)=C%12C)=C%12CO%11)C%10=C9O"]

Rhizolutin_smiles = "C/C1=C\[C@H](O2)[C@@](/C=C/C(C)=C\[C@]3([H])[C@]1([H])C[C@H](CC)OC3=O)([H])C[C@H](O)CC2=O"

penicimeroterpenoids_smiles_list = ["O=C1[C@@](C)(O)C([C@@]2(C(OC)=O)[C@@]3(C)C[C@H](OC4=O)[C@]5([H])C(C)(C)[C@@H](OC(C)=O)CC[C@@]54[C@]3([H])[C@@H]1C(C)=C2C)=O", \
              "C[C@]12C[C@H](OC3=O)[C@]4([H])C(C)(C)[C@@H](OC(C)=O)CC[C@@]43[C@]1([H])C=C(C)[C@]5(C)[C@@]2(C(OC)=O)C([C@]5(O)C)=O", \
              "C[C@]12C[C@H](OC3=O)[C@]4([H])C(C)(C)[C@@H](OC(C)=O)CC[C@@]43[C@]1([H])C=C(C)[C@]5(C)[C@@]2(C(OC)=O)[C@@](O)(C)C5=O"]

BosamycinA_smiles = "N[C@H](CC1=CC=C(O)C=C1)C(N[C@@H](CC(C)C)C(N[C@H]([C@H](C(O)=O)O)C(N[C@@H](CO)C(N[C@H](CC2=CC=C(O)C=C2OC)C(N[C@@H](CC(C)C)C(NCC(O)=O)=O)=O)=O)=O)=O)=O"

phakefustatin_smiles = "O=C(N1[C@@H](CCC1)C(N[C@@H](CC2=CC=CC=C2)C(N3)=O)=O)[C@H](C(C)C)NC([C@H]4N(C([C@H]5N(C([C@H](CCCNC(N)=N)NC([C@@H]3CC(C6=CC=CC=C6N)=O)=O)=O)CCC5)=O)CCC4)=O"

SI_smiles_list = ["OC[C@@H](O1)[C@@H](O)[C@H](O)[C@@H](O)[C@@H]1OCCC2=CC=C(O)C=C2", "OC(CCC/C=C1C[C@@H]2[C@H]([C@@H](C[C@@H]2O/1)O)/C=C/[C@H](CCCCC)O)=O", \
               "CC[C@H]1[C@@H](C)C(C(C)=C([C@H]([C@@H](O)CC)C)O1)=O", "C[C@@H]([C@H]1CC[C@@H]2[C@]1(C)CC[C@H]3[C@H]2CC=C4[C@]3(C)CC[C@H](O)C4)CCCC(C)C", \
               "OC/C=C(C)/CC/C=C(C)/CC/C=C(C)\C", "O[C@@H]1[C@H](CC[C@@H](C1)C)C(C)C", "O=CC1=CC=CC=C1", \
               "CC1C(=O)NC(C(=O)NC2CSSCC3C(=O)NC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NC(CSSCC(C(=O)NC(CSSCC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NCC(=O)N1)CCCCN)CCCCN)N)C(=O)NC(C(=O)NCC(=O)NC(C(=O)N3)CO)C(C)O)NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC2=O)CO)CCCNC(=N)N)CC(C)C)CCSC)CC4=CC=C(C=C4)O)CC(=O)O)C(=O)N)CCCCN)CO)CCCNC(=N)N)CCCCN"]

In [36]:
frame = pd.read_pickle(folder + "NPA_ML-with-prop.pkl")

In [37]:
MAP4 = MAP4Calculator(dimensions=1024)
def calc_map4(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smiles = Chem.MolToSmiles(mol, isomericSmiles=False)
    mol = Chem.MolFromSmiles(smiles)
    map4 = MAP4.calculate(mol)
    return np.array(map4)

def map4_kernel_SVM(a, b=None):
    if b is None:
        b=a
    JS_all_pairs = np.zeros((len(a),len(b)))
    for i,fp1 in enumerate(a):
        for j,fp2 in enumerate(b):
            JS_all_pairs[i,j] = np.float(np.count_nonzero(fp1 == fp2)) / np.float(len(fp1))
    return JS_all_pairs

def map4_kernel_NN(fp1, fp2):
    return 1- np.float(np.count_nonzero(fp1 == fp2)) / np.float(len(fp1))


def distance(a, b):
    """Estimates the Jaccard distance of two binary arrays based on their hashes.

Arguments:
  a {numpy.ndarray} -- An array containing hash values.
  b {numpy.ndarray} -- An array containing hash values.

Returns:
  float -- The estimated Jaccard distance.
"""

    # The Jaccard distance of Minhashed values is estimated by
    return 1.0 - np.float(np.count_nonzero(a == b)) / np.float(len(a))

def find_map_seqNN(fp, dataframe = frame[frame.Set=="training"]):
    best_dist = float("inf")
    dists = dataframe["MAP4"].map(lambda fp2 : distance(fp,fp2))
    NNi = np.argmin(dists)
    best_dist = dists.iloc[NNi]
    NN = dataframe["NPAID"].iloc[NNi]
    return best_dist, NN

In [38]:
SVM = joblib.load(folder + "SVM-NPA.pkl")
kNN = joblib.load(folder + "NN-NPA.pkl")

In [39]:
epicospirocins_map4_list = [calc_map4(smiles) for smiles in epicospirocins_smiles_list]
penicimeroterpenoids_map4_list = [calc_map4(smiles) for smiles in penicimeroterpenoids_smiles_list]
Rhizolutin_map4 = calc_map4(Rhizolutin_smiles)
BosamycinA_map4 = calc_map4(BosamycinA_smiles)
phakefustatin_map4 = calc_map4(phakefustatin_smiles)
SI_map4_list = [calc_map4(smiles) for smiles in SI_smiles_list]

epicospirocins_NN_dist_list = [find_map_seqNN(fp) for fp in epicospirocins_map4_list]
penicimeroterpenoids_NN_dist_list = [find_map_seqNN(fp) for fp in penicimeroterpenoids_map4_list]
Rhizolutin_NN_dist = find_map_seqNN(Rhizolutin_map4)
BosamycinA_NN_dist = find_map_seqNN(BosamycinA_map4)
phakefustatin_NN_dist = find_map_seqNN(phakefustatin_map4)
SI_NN_dist_list = [find_map_seqNN(fp) for fp in SI_map4_list]

In [40]:
epicospirocins_NN_dist_list

[(0.658203125, 'NPA024935'),
 (0.4921875, 'NPA024935'),
 (0.7490234375, 'NPA024935'),
 (0.615234375, 'NPA024935')]

In [41]:
penicimeroterpenoids_NN_dist_list

[(0.6318359375, 'NPA016454'),
 (0.466796875, 'NPA016454'),
 (0.5361328125, 'NPA016454')]

In [42]:
Rhizolutin_NN_dist

(0.802734375, 'NPA009354')

In [43]:
BosamycinA_NN_dist

(0.7666015625, 'NPA010987')

In [44]:
phakefustatin_NN_dist

(0.6787109375, 'NPA022212')

In [45]:
SI_NN_dist_list

[(0.751953125, 'NPA016219'),
 (0.78515625, 'NPA005601'),
 (0.8154296875, 'NPA004018'),
 (0.369140625, 'NPA018196'),
 (0.640625, 'NPA013150'),
 (0.501953125, 'NPA012557'),
 (0.734375, 'NPA008069'),
 (0.75390625, 'NPA020589')]

In [46]:
pred_epicospirocins_SVM = SVM.predict_proba(epicospirocins_map4_list)
pred_penicimeroterpenoids_SVM = SVM.predict_proba(penicimeroterpenoids_map4_list)
pred_Rhizolutin_SVM = SVM.predict_proba([Rhizolutin_map4])
pred_BosamycinA_SVM = SVM.predict_proba([BosamycinA_map4])
pred_phakefustatin_SVM = SVM.predict_proba([phakefustatin_map4])
pred_SI_SVM = SVM.predict_proba(SI_map4_list)

pred_epicospirocins_NN = kNN.predict_proba(epicospirocins_map4_list)
pred_penicimeroterpenoids_NN = kNN.predict_proba(penicimeroterpenoids_map4_list)
pred_Rhizolutin_NN = kNN.predict_proba([Rhizolutin_map4])
pred_BosamycinA_NN = kNN.predict_proba([BosamycinA_map4])
pred_phakefustatin_NN = kNN.predict_proba([phakefustatin_map4])
pred_SI_NN = kNN.predict_proba(SI_map4_list)

In [47]:
print(pred_epicospirocins_SVM) # predicted fungal origin

[[0.98769655 0.01230345]
 [0.98692661 0.01307339]
 [0.98614541 0.01385459]
 [0.97942783 0.02057217]]


In [48]:
print(pred_epicospirocins_NN) # predicted fungal origin

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [49]:
pred_Rhizolutin_SVM

array([[0.83459318, 0.16540682]])

In [50]:
pred_Rhizolutin_NN

array([[0.85714286, 0.14285714]])

In [51]:
print(pred_penicimeroterpenoids_SVM) # predicted fungal origin

[[0.99775061 0.00224939]
 [0.99895939 0.00104061]
 [0.99703121 0.00296879]]


In [52]:
print(pred_penicimeroterpenoids_NN) # predicted fungal origin

[[1. 0.]
 [1. 0.]
 [1. 0.]]


In [53]:
print(pred_BosamycinA_SVM)

[[0.04215834 0.95784166]]


In [54]:
print(pred_BosamycinA_NN)

[[0. 1.]]


In [55]:
print(pred_phakefustatin_SVM) # predicted bacterial origin

[[0.12203697 0.87796303]]


In [56]:
print(pred_phakefustatin_NN) # predicted fungal origin

[[0.57142857 0.42857143]]


In [57]:
print(pred_SI_SVM)

[[0.75137514 0.24862486]
 [0.94108369 0.05891631]
 [0.16166844 0.83833156]
 [0.92935664 0.07064336]
 [0.92020936 0.07979064]
 [0.95917085 0.04082915]
 [0.30263708 0.69736292]
 [0.01037179 0.98962821]]


In [58]:
print(pred_SI_NN)

[[0.42857143 0.57142857]
 [0.85714286 0.14285714]
 [0.14285714 0.85714286]
 [1.         0.        ]
 [0.57142857 0.42857143]
 [1.         0.        ]
 [0.42857143 0.57142857]
 [0.         1.        ]]
